# Normality Tests

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from constants.real_time_constants import SLIDING_WINDOW_LEN, STRIDE
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset
from data.sliding_window import SlidingWindowDataset

## pure data
id_train_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset",
    split_input_and_output_ids=False,
)
id_test_base_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset",
    split_input_and_output_ids=True,
)
ood_test_base_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

print(len(id_train_base_dataset))
## dataset that takes chunks of 120 tokens out of the above datasets
id_train_dataset = SlidingWindowDataset(
    base_dataset=id_train_base_dataset,
    name="id_train_dataset_no_distinction",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)
id_test_dataset = SlidingWindowDataset(
    base_dataset=id_test_base_dataset,
    name="id_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)   
ood_test_dataset = SlidingWindowDataset(
    base_dataset=ood_test_base_dataset,
    name="ood_test_dataset",
    k=SLIDING_WINDOW_LEN,
    stride=STRIDE,
)

print("Length of id_train_dataset:", len(id_train_dataset))
print("Length of id_test_dataset:", len(id_test_dataset))
print("Length of ood_test_dataset:", len(ood_test_dataset))


/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detected 0 bad samples
Detected 0 bad samples
432
Length of id_train_dataset: 54512
Length of id_test_dataset: 2298
Length of ood_test_dataset: 144345


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    try:
        events = get_readable_events(input_ids)
    except Exception as e:
        print(f"Error: {e}")
        print(f"Input IDs: {input_ids}")
        continue
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 50)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df.to_csv("events.csv", index=False)
df

dataset id_train                                    id_test               ...  \
key         misc  onset   dur inst pitch  antic vel    misc  onset   dur  ...   
0             AR                                         AR               ...   
1                 17.61   0.1    0    G2  False              15.53  0.09  ...   
2                 17.62  0.06    0    G4  False               15.7  0.04  ...   
3                 17.76  0.05    0    A4  False              15.86  0.43  ...   
4                 17.77  0.07    0    A2  False              15.87  0.21  ...   
5                 17.89  0.09    0    C3  False              15.87  0.33  ...   
6                 17.89  0.04    0    C5  False              16.05  0.11  ...   
7                 18.04  0.11    0    D2  False              16.19  0.17  ...   
8                 18.05   0.1    0    D4  False              16.36  0.14  ...   
9                 18.18  0.11    0    A2  False              16.37  0.17  ...   
10                18.18  0.06    0    A4  False              16.37  0.12  ...   
11                18.33  0.03    0    D5  False              16.53  0.11  ...   
12                18.33  0.13    0    D3  False              16.69   0.1  ...   
13                18.46  0.07    0    G2  False              16.88  0.13  ...   
14                18.48  0.04    0    G4  False              20.18  0.05  ...   
15                18.61  0.04    0    A4  False              20.18  0.05  ...   
16                18.62  0.06    0    A2  False              20.18  0.05  ...   
17                18.75  0.04    0    D5  False              20.19  0.05  ...   
18                18.76  0.07    0    C3  False               20.2  0.04  ...   
19                18.89  0.15    0    D2  False              20.36   0.2  ...   
20                 18.9  0.07    0    D4  False              20.36  0.22  ...   
21                19.04  0.09    0    A2  False              20.36  0.08  ...   
22                19.04  0.07    0    A4  False              20.37  0.19  ...   
23                19.17  0.11    0    D3  False              20.37  0.13  ...   
24                19.18  0.03    0    D5  False              20.53  0.05  ...   
25                19.32  0.06    0    G4  False              20.71  0.06  ...   
26                19.32  0.09    0    G2  False              20.72  0.05  ...   
27                19.46  0.06    0    A4  False              20.72  0.04  ...   
28                19.47  0.08    0    A2  False              20.73  0.05  ...   
29                19.59  0.09    0    C3  False              20.73  0.04  ...   
30                 19.6  0.04    0    C5  False              20.73  0.03  ...   
31                19.63  0.03    0    D5  False              20.89  0.22  ...   
32                19.73  0.14    0    D2  False              20.89  0.24  ...   
33                19.75  0.08    0    A4  False              20.89  0.24  ...   
34                19.87  0.12    0    A2  False              20.89  0.08  ...   
35                19.89  0.08    0    D5  False               20.9  0.22  ...   
36                 19.9  0.04    0    C5  False               20.9  0.24  ...   
37                20.01  0.11    0    D3  False              21.08  0.07  ...   
38                20.02  0.06    0    A5  False              21.25  0.06  ...   
39                20.16  0.09    0    G2  False              21.26  0.06  ...   
40                20.17  0.06    0    D5  False              21.26  0.05  ...   

dataset                  ood_test                                    
key     pitch  antic vel     misc onset   dur inst pitch  antic vel  
0                              AR                                    
1          C5  False               3.33  0.12    0   G#2  False      
2         A#4  False               3.34  0.22    0    B5  False      
3          D4  False               3.53  0.06    0    F5  False      
4          C5  False               3.54  0.04    0    D5  False      
5          G3  False      

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value 1.0
{'skewness': np.float64(171953.956), 'kurtosis': np.float64(-0.508), 'skewness_p': 0.332, 'kurtosis_p': 0.612, 'royston_p': 0.685, 'hz_p_value': 1.0}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

Representations already exist. Loading from disk.


Loading layers from disk:   4%|▍         | 1/25 [00:00<00:04,  5.69it/s]

Loading layer 0 from disk.
Loading layer 1 from disk.


Loading layers from disk:  12%|█▏        | 3/25 [00:00<00:03,  5.95it/s]

Loading layer 2 from disk.
Loading layer 3 from disk.


...

Loading layer 21 from disk.
Loading layer 22 from disk.


Loading layers from disk:  96%|█████████▌| 24/25 [00:04<00:00,  5.75it/s]

Loading layer 23 from disk.
Loading layer 24 from disk.


Loading layers from disk: 100%|██████████| 25/25 [00:04<00:00,  5.54it/s]


{0: array([[ 0.00084921, -0.00873748,  0.00461115, ...,  0.04268945,
          0.040856  ,  0.03629989],
        [ 0.00109164, -0.00838182,  0.00576359, ...,  0.04032072,
          0.04039811,  0.03468793],
        [-0.00091089, -0.00910859,  0.00637074, ...,  0.04147919,
          0.04152303,  0.03586728],
        ...,
        [ 0.00225269,  0.00175249, -0.0063197 , ...,  0.03006084,
          0.03870829,  0.03935357],
        [ 0.00194811,  0.00283337, -0.00641365, ...,  0.02923518,
          0.03886895,  0.03997277],
        [ 0.00233792,  0.00218778, -0.00481814, ...,  0.03030348,
          0.03500358,  0.03929587]], shape=(54512, 2048), dtype=float32),
 1: array([[ 0.01601014,  0.002558  , -0.03870387, ...,  0.10005309,
          0.10067132,  0.12693486],
        [ 0.01176107,  0.01705838, -0.03209689, ...,  0.09128595,
          0.09336561,  0.10297351],
        [ 0.00699234,  0.01631417, -0.03805479, ...,  0.09237039,
          0.09098511,  0.1053161 ],
        ...,
        [ 0.

In [8]:
import numpy as np
import pandas as pd
from extract_layers.extract_layers_main import extract_representations
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_data = extract_representations(
        model,
        id_train_dataloader,
        pooling_function,
        [layer_idx],
    )
    layer_data=layer_data[layer_idx][..., :10]
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (54512, 10)
Running test mardia
  Subsampling to 5000 samples and 10 features for testing
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz
  Subsampling to 5000 samples and 10 features for testing
p_value <0.001


Processing layers:   4%|▍         | 1/25 [00:03<01:14,  3.09s/it]

Representations already exist. Loading from disk.


Loading layers from disk: 100%|██████████| 1/1 [00:00<00:00,  5.73it/s]


...

Representations already exist. Loading from disk.


Loading layer 24 from disk.


Loading layers from disk: 100%|██████████| 1/1 [00:00<00:00,  4.28it/s]


Processing layer 24
Layer data shape: (54512, 10)
Running test mardia
  Subsampling to 5000 samples and 10 features for testing
Running test royston
  Subsampling to 1000 samples and 10 features for testing
Running test hz
  Subsampling to 5000 samples and 10 features for testing
p_value <0.001


Processing layers: 100%|██████████| 25/25 [01:18<00:00,  3.14s/it]


In [9]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,21830.164,100.302,0.001,0.001,0.001,0.001
1,4244.359,14.743,0.001,0.001,0.001,0.001
2,4529.486,17.732,0.001,0.001,0.001,0.001
3,3736.013,13.806,0.001,0.001,0.001,0.001
4,4182.297,20.122,0.001,0.001,0.001,0.001
5,3301.936,11.206,0.001,0.001,0.001,0.001
6,3578.704,13.592,0.001,0.001,0.001,0.001
7,4060.348,15.463,0.001,0.001,0.001,0.001
8,3137.495,13.534,0.001,0.001,0.001,0.001
9,2798.354,9.520,0.001,0.001,0.001,0.001


In [10]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)